In [2]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [10]:
# Create drive to access Google Drive
gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
gauth.LoadClientConfigFile("credentials.json")
drive = GoogleDrive(gauth)

In [11]:
download = drive.CreateFile({'id': '1BZOv422XJvxFUnGh-0xVeSvgFgqVY45q'})

In [12]:
download.GetContentFile('train_LbELtWX.zip')


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=772264689031-rqes5qk1co2il7mnl5cu00h0vrnu6d4h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [34]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image

In [76]:
train = pd.read_csv('train.csv')

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [37]:
train_images = []
for i in tqdm(range(train.shape[0])):
    img = Image.open('train/'+train['id'][i].astype('str')+'.png')
    img = image.img_to_array(img)
    img = img/255
    train_images.append(img)
X = np.array(train_images)

100%|██████████| 60000/60000 [00:20<00:00, 2958.00it/s]


In [43]:
len(y)

60000

In [86]:
len(X[0]) # Len 28
len(X[0][0]) # Len 28
len(X[0][0][0]) # Len 4

array([0., 0., 0., 1.], dtype=float32)

In [44]:
y = train['label'].values
y = to_categorical(y)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.2)

In [60]:
len(X_test)

12000

In [71]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,4)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [72]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [111]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 80s 2ms/step - loss: 0.1699 - accuracy: 0.9365 - val_loss: 0.2213 - val_accuracy: 0.9296
Epoch 2/10
48000/48000 [==============================] - 82s 2ms/step - loss: 0.1700 - accuracy: 0.9357 - val_loss: 0.2250 - val_accuracy: 0.9258
Epoch 3/10
48000/48000 [==============================] - 82s 2ms/step - loss: 0.1582 - accuracy: 0.9406 - val_loss: 0.2216 - val_accuracy: 0.9261
Epoch 4/10
48000/48000 [==============================] - 79s 2ms/step - loss: 0.1509 - accuracy: 0.9431 - val_loss: 0.2439 - val_accuracy: 0.9242
Epoch 5/10
48000/48000 [==============================] - 83s 2ms/step - loss: 0.1493 - accuracy: 0.9433 - val_loss: 0.2223 - val_accuracy: 0.9271
Epoch 6/10
48000/48000 [==============================] - 85s 2ms/step - loss: 0.1410 - accuracy: 0.9472 - val_loss: 0.2341 - val_accuracy: 0.9225
Epoch 7/10
48000/48000 [==============================] - 86s 2ms/st

In [88]:
# download = drive.CreateFile({'id': '1KuyWGFEpj7Fr2DgBsW8qsWvjqEzfoJBY'})
# download.GetContentFile('test_ScVgIM0.zip')
# !unzip test_ScVgIM0.zip

In [107]:
test_df = pd.read_csv('test.csv')

In [92]:
test.shape

(10000, 1)

In [93]:
test

,id
0,60001
1,60002
2,60003
3,60004
4,60005
...,...
9995,69996
9996,69997
9997,69998
9998,69999


In [102]:
test_images = []
for i in tqdm(range(test_df.shape[0])):
    img = Image.open('test/'+test_df['id'][i].astype('str')+'.png')
    img = image.img_to_array(img)
    img = img/255
    test_images.append(img)
test = np.array(test_images)

100%|██████████| 10000/10000 [00:03<00:00, 3041.86it/s]


In [112]:
predictions = model.predict_classes(test)

In [113]:
predictions

array([9, 2, 1, ..., 8, 1, 5])

In [114]:
test_df['label'] = predictions
test_df

,id,label
0,60001,9
1,60002,2
2,60003,1
3,60004,1
4,60005,6
...,...,...
9995,69996,9
9996,69997,1
9997,69998,8
9998,69999,1


In [115]:
test_df.to_csv('submission_cnn.csv', header=True, index=False)